# My custom env

Here I can easily fix what is non conform to Ray

In [ ]:
import numpy as np
import functools
import gymnasium as gym
from gymnasium import spaces
from pettingzoo import ParallelEnv
import pygame 
import os
from collections import defaultdict
import random

SPRITES_DIR = "./sprites"


class MyGridWorld(ParallelEnv):
    metadata = {"render_modes": ["human"], "name": "custom_grid_v0"}

    def __init__(self, render_mode=None, n_agents = 2, grid_size=15):
        if grid_size<8:
            print("Error, need to insert a grid size greater or equal to 8")
        self.grid_size = grid_size
        self.render_mode = render_mode

        ######## Agents, fixed components, and forbidden positions
        self.possible_agents = [f"agent{i}" for i in range(1, n_agents+1)]
        self.agents = self.possible_agents[:]
        self.gate_open = False

        self.fixed_components = {
            "button":  {"pos": np.array([self.grid_size//2+2, 1+3+2]), "file": "button.png"},
            "gate_open":  {"pos": np.array([self.grid_size//2, 1+3]), "file": "gate_open.png"},
            "gate_close":  {"pos": np.array([self.grid_size//2, 1+3]), "file": "gate_close.png"}   
        }
        self.button_pos = self.fixed_components["button"]["pos"]
        self.gate_pos = self.fixed_components["gate_open"]["pos"] 

        self.x_range = (1, self.grid_size-1-1)
        self.y_range = (1+3+1, self.grid_size-1-1)
        self.forbidden_position = {tuple(self.button_pos)}    

        self.max_cycles = 100
        self.current_cycles = 0
        ######## Pygame graphic configuration 
        self.window_size = 810
        self.cell_size = self.window_size // self.grid_size
        self.window = None
        self.clock = None
        
        # Walls
        self.grid_map = np.zeros((self.grid_size, self.grid_size))
        self.grid_map[0, :] = 1
        self.grid_map[:, 0] = 1
        self.grid_map[-1, :] = 1
        self.grid_map[:, -1] = 1
        self.grid_map[ :, 1+3] = 1

        # Sprites 
        self.agent_sprites = {}
        self.component_sprites = {}

        ######## Action space and observation space
        # Five possible actions in each grid: stay(0), up(1), down(2), left(3), right(4)
        self.action_spaces = {a: spaces.Discrete(5) for a in self.possible_agents}
        OBSERVATION_DIM = 2*(len(self.agents)-1)+2*2+1
        self.observation_spaces = {
            a: spaces.Box(low=-(self.grid_size-1), high=self.grid_size-1, shape=(OBSERVATION_DIM,), dtype=np.float32) 
            for a in self.possible_agents
        }


    # Boilerplate PettingZoo
    def observation_space(self, agent): return self.observation_spaces[agent]
    def action_space(self, agent): return self.action_spaces[agent]

    '''
    Step in the environment
    '''
    def step(self, actions):
        if not self.agents: return {}, {}, {}, {}, {}
        self.current_cycles += 1
        
        rewards = {a: -1 for a in self.agents}
        terminations = {a: False for a in self.agents}
        truncations = {a: False for a in self.agents}
        infos = {a: {} for a in self.agents}
        
        desired_positions = {}      
        button_pressed = False
        agents_desire_gate = []
        for agent, action in actions.items():
            current_pos = self.agent_positions[agent].copy()
            target_pos = current_pos.copy()

            if action == 1: target_pos[1] -= 1 
            elif action == 2: target_pos[1] += 1
            elif action == 3: target_pos[0] -= 1
            elif action == 4: target_pos[0] += 1

            # Monitor button, gate and walls
            is_wall = self.grid_map[target_pos[0], target_pos[1]] == 1
            is_gate = (target_pos == self.gate_pos).all()
            is_button = (target_pos == self.button_pos).all()
            
            if is_button:
                button_pressed = True
            if is_gate:
                agents_desire_gate.append((agent, current_pos))
            
            if is_wall and not is_gate:
                desired_positions[agent] = current_pos 
            else:
                desired_positions[agent] = target_pos
                        
        # If gate was open remove a wall, if gate was closed update the position of those who wanted to cross it
        self.gate_open = button_pressed
        if self.gate_open:
            self.grid_map[self.gate_pos[0], self.gate_pos[1]] = 0
        else:
            self.grid_map[self.gate_pos[0], self.gate_pos[1]] = 1
            for a in agents_desire_gate:
                agent = a[0]
                current_pos = a[1]
                desired_positions[agent] = current_pos 

        # Check for conflicts (more than one agents have the same desired position)
        final_positions = self.agent_positions.copy()
        target_counts = defaultdict(list)         # Key: tuple(x, y) of desired positions, Value: list of agents desiring it
        for agent, pos in desired_positions.items():
            pos_tuple = tuple(pos)
            target_counts[pos_tuple].append(agent)

        # Solve eventual conflicts
        for pos_tuple, agents_at_target in target_counts.items():      
            # Case 1: no contended position
            if len(agents_at_target) == 1:
                agent = agents_at_target[0]
                final_positions[agent] = desired_positions[agent]
            
            # Case 2: contended position
            else:
                one_agent_already_here_not_moving = None
                for agent in agents_at_target:
                    if tuple(self.agent_positions[agent]) == pos_tuple:
                        one_agent_already_here_not_moving = agent
                        break
                
                winning_agent = one_agent_already_here_not_moving if one_agent_already_here_not_moving else random.choice(agents_at_target)
                final_positions[winning_agent] = desired_positions[winning_agent]
                
                agents_at_target.remove(winning_agent)
                for losing_agent in agents_at_target[:]:
                    final_positions[losing_agent] = self.agent_positions[losing_agent] 
        self.agent_positions = final_positions

        # Negative reward if an agent is on the bottom area and is not pressing the button
        agents_upper_area = 0
        for a, pos in self.agent_positions.items():
            if pos[1]<4:
                rewards[a] = +1
                agents_upper_area+=1
            if (pos[0]==self.button_pos[0] and pos[1]==self.button_pos[1]):
                rewards[a] = +1

       
        if agents_upper_area == len(self.agents)-1:
            rewards = {a: +100 for a in self.agents}
            terminations = {a: True for a in self.agents}
        if self.current_cycles >= self.max_cycles:
            truncations= {a: True for a in self.agents}
            rewards = {a: -100 for a in self.agents}

        if self.render_mode == "human":
            self.render()

        final_obs = self.gather_observations()
        self.agents = [a for a in self.agents if not terminations[a]]

        return final_obs, rewards, terminations, truncations, infos
   
    '''
    Determines initial condition of the simulation
    '''
    def generate_valid_position(self):
        while True:
            x = np.random.randint(self.x_range[0], self.x_range[1])
            y = np.random.randint(self.y_range[0], self.y_range[1])
            new_position = tuple((x, y))

            if new_position not in self.forbidden_position:
                return new_position
            
    def generate_set_initial_positions(self):
        set_initial_positions = set()
        while len(set_initial_positions)<len(self.possible_agents):
            new_pos = self.generate_valid_position()
            set_initial_positions.add(new_pos)
        return set_initial_positions
    
    def generate_test_values(self, agent_num):
        if agent_num==1:
            return[7, 9]
        else:
            return [9,13]

    def reset(self, seed=None, options=None):
        self.agents = self.possible_agents[:]
        set_initial_positions = list(self.generate_set_initial_positions())
        self.current_cycles = 0

        self.agent_positions = {}
        for i, agent_id in enumerate(self.agents):
            self.agent_positions[agent_id] = np.array(set_initial_positions[i])
        
        # To visualize the reset position if "human" mode on
        if self.render_mode == "human":
            self.render()         

        return self.gather_observations(), {a: {} for a in self.agents}
    
    '''
    Observation: [my_cur_pos, (other_cur) x number of other agents, button_pos, gate_pos, gate_open (1|0)]    '''
    def gather_observations(self):
        gate_status_info = np.array([int(self.gate_open)], dtype=np.float32)
        observations = {}
        for observing_agent in self.agents:
            obs_segments = []
            observing_agent_pos = self.agent_positions[observing_agent]
            
            for other_agent in self.agents:
                if other_agent != observing_agent:
                    obs_segments.append(self.agent_positions[other_agent] - observing_agent_pos)
            obs_segments.append(self.button_pos- observing_agent_pos)
            obs_segments.append(self.gate_pos -observing_agent_pos)
            obs_segments.append(gate_status_info)
            observations[observing_agent] = np.concatenate(obs_segments, dtype=np.float32)
        return observations

    '''
    Functions for Sprite rendering and loading
    '''    
    def _create_missing_sprite(self, text, bg_color, border_color = (0,0,0)):  
        error_sprite = pygame.Surface((self.cell_size, self.cell_size), pygame.SRCALPHA)
        error_sprite.fill((0, 0, 0, 0))
        center = (self.cell_size // 2, self.cell_size // 2)
        radius = int(self.cell_size * 0.4)
        pygame.draw.circle(error_sprite, bg_color, center, radius) 
        pygame.draw.circle(error_sprite, border_color, center, radius, 2)
        
        # Add text label
        try:
            font_size = int(self.cell_size * 0.22)
            font = pygame.font.Font(None, font_size)   
            text_color = (0, 0, 0) if sum(bg_color) > 300 else (255, 255, 255) 
            text_surface = font.render(text, True, text_color)
            text_rect = text_surface.get_rect(center=center)
            error_sprite.blit(text_surface, text_rect)
            
        except Exception as e:
            print(f"WARNING: Error while writing text inside missing sprite {text}. A missing sprite without text label will be used for this simulation.")
            pass
            
        return error_sprite
    
    def _load_and_scale_sprite(self, filename, component_name):
        path = os.path.join(SPRITES_DIR, filename)
    
        try:
            image = pygame.image.load(path).convert_alpha() 
            scaled_size = int(self.cell_size )
            return pygame.transform.scale(image, (scaled_size, scaled_size))
            
        except (FileNotFoundError, pygame.error) as e:
            print(f"WARNING! Sprite image {path} not found. Using default sprite for this simulation.")
    
            text = component_name.upper() if component_name else "ERROR"
            bg_color = (100, 100, 255) if "AGENT" in text else (128,128,128) # Blue agent, Gray fixed components
            return self._create_missing_sprite(text, bg_color)

    def render(self):
        if self.render_mode is None:
            return

        if self.window is None:
            pygame.init()
            pygame.display.init()
            pygame.font.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
            
            for agent_name in self.agents:
               self.agent_sprites[agent_name] = self._load_and_scale_sprite(".png", agent_name)

            for name, data in self.fixed_components.items():
                self.component_sprites[name] = self._load_and_scale_sprite(data["file"], name)

                 
        if self.clock is None:
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((220, 220, 220)) 
        pix_square_size = self.cell_size 
        sprite_offset = (pix_square_size - int(pix_square_size * 0.8)) // 2

        # Render walls
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                if self.grid_map[x, y] == 1:
                    pygame.draw.rect(
                        canvas, 
                        (50, 50, 50),
                        pygame.Rect(x * pix_square_size, y * pix_square_size, pix_square_size, pix_square_size)
                    )

        # Render fixed components and agents
        for name, data in self.fixed_components.items():
            if self.gate_open and name == "gate_close":
                continue
            pos = data["pos"]
            x_coord = pos[0] * pix_square_size + sprite_offset
            y_coord = pos[1] * pix_square_size + sprite_offset
            sprite = self.component_sprites[name]
            canvas.blit(sprite, (x_coord, y_coord))

        for agent in self.agents:
            pos = self.agent_positions[agent]            
            x_coord = pos[0] * pix_square_size + sprite_offset
            y_coord = pos[1] * pix_square_size + sprite_offset
            
            sprite = self.agent_sprites[agent]
            canvas.blit(sprite, (x_coord, y_coord))

        # Render the grid
        for x in range(self.grid_size + 1):
            pygame.draw.line(
                canvas, 0, (0, pix_square_size * x), (self.window_size, pix_square_size * x), width=2
            )
            pygame.draw.line(
                canvas, 0, (pix_square_size * x, 0), (pix_square_size * x, self.window_size), width=2
            )

        # Display the render
        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.update()
        self.clock.tick(4) # FPS

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


/home/terra/anaconda3/envs/MARL/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.tune.registry import register_env

def env_creator(config):
    return ParallelPettingZooEnv(
        MyGridWorld(render_mode=None)
    )

register_env("my_gridworld", env_creator)

## Configure PPO
Set my custom environment

Set policy

Set training parameters

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig

config = (
    PPOConfig()
    .environment(env="my_gridworld")
    .multi_agent(
        policies={"shared_policy"},
        policy_mapping_fn=lambda agent_id, *args, **kwargs: "shared_policy",
    )
    .env_runners(num_env_runners=1) 
    .training(
        lr=2e-4,
        train_batch_size_per_learner=2000,
        num_epochs=10,
    )
    .api_stack(
        enable_rl_module_and_learner=False,
        enable_env_runner_and_connector_v2=False
    )
)
from ray.rllib.algorithms.ppo import PPOConfig

ppo = config.build_algo()

/home/terra/anaconda3/envs/MARL/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:527: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/terra/anaconda3/envs/MARL/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/terra/anaconda3/envs/MARL/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is depreca

## Training

In [4]:
for i in range(50):
    result = ppo.train()
    print(
        f"Iter:{i}:\n Rew:{result['env_runners']['episode_return_mean']}, len episode:{result['env_runners']['episode_len_mean']}"
    )

2025-12-25 20:04:11,506	WARNING sgd.py:55 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Iter:0:
 Rew:-393.3, len episode:100.0
Iter:1:
 Rew:-390.35, len episode:100.0
Iter:2:
 Rew:-387.3666666666667, len episode:100.0


(pid=gcs_server) [2025-12-25 20:04:27,854 E 354373 354373] (gcs_server) gcs_server.cc:303: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(raylet) [2025-12-25 20:04:29,022 E 354487 354487] (raylet) main.cc:1032: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(RolloutWorker pid=354544) [2025-12-25 20:04:29,947 E 354544 354629] core_worker_process.cc:842: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
[2025-12-25 20:04:30,428 E 354333 354540] core_worker_process.cc:842: Failed to establish connection to the metrics exporter agent. Metrics

Iter:3:
 Rew:-380.525, len episode:100.0
Iter:4:
 Rew:-370.1, len episode:100.0
Iter:5:
 Rew:-350.86, len episode:100.0
Iter:6:
 Rew:-330.16, len episode:100.0
Iter:7:
 Rew:-307.52, len episode:100.0
Iter:8:
 Rew:-286.66, len episode:100.0
Iter:9:
 Rew:-269.2, len episode:100.0
Iter:10:
 Rew:-256.88, len episode:100.0
Iter:11:
 Rew:-246.06, len episode:100.0
Iter:12:
 Rew:-237.28, len episode:100.0
Iter:13:
 Rew:-229.74, len episode:100.0
Iter:14:
 Rew:-225.1, len episode:100.0
Iter:15:
 Rew:-221.4, len episode:100.0
Iter:16:
 Rew:-217.82, len episode:100.0
Iter:17:
 Rew:-215.82, len episode:100.0
Iter:18:
 Rew:-214.2, len episode:100.0
Iter:19:
 Rew:-212.9, len episode:100.0
Iter:20:
 Rew:-211.62, len episode:100.0
Iter:21:
 Rew:-211.5, len episode:100.0
Iter:22:
 Rew:-210.76, len episode:100.0
Iter:23:
 Rew:-211.58, len episode:100.0
Iter:24:
 Rew:-211.68, len episode:100.0
Iter:25:
 Rew:-211.72, len episode:100.0
Iter:26:
 Rew:-211.64, len episode:100.0
Iter:27:
 Rew:-211.3, len epi

## Save the agents model in order to reuse it

In [16]:
import os
current_dir = os.getcwd()
model_dir = os.path.join(current_dir, "models")
checkpoint_name = os.path.join(model_dir, "ck2_3agents")
ppo.save(checkpoint_name)

print(f"Model saved into {checkpoint_name}")

Model saved into /home/terra/Desktop/unimore/distributed_project/src/policies/PPO/models/ck2_3agents


## Load an agent

In [31]:
from ray.rllib.algorithms.ppo import PPO
import os
current_dir = os.getcwd()
model_dir = os.path.join(current_dir, "models")
checkpoint_name = os.path.join(model_dir, "ck2_3agents")

ppo = PPO.from_checkpoint(checkpoint_name)

(RolloutWorker pid=279765) /home/terra/anaconda3/envs/MARL/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(RolloutWorker pid=279765)   from pkg_resources import resource_stream, resource_exists
2025-12-25 19:00:25,738	WARNING util.py:61 -- Install gputil for GPU system monitoring.


## Test the agent

In [5]:
POLICY_ID = "shared_policy" 

env = MyGridWorld(grid_size=15, render_mode="human")
obs, _ = env.reset()

i = 0
while i<3:
    actions = {}
    for agent_id, agent_obs in obs.items():
        actions[agent_id] = ppo.compute_single_action(
            agent_obs, 
            policy_id=POLICY_ID,
            explore=False
        )
    
    obs, rewards, terminations, truncations, infos = env.step(actions)

    if any(terminations.values()) or any(truncations.values()):
        print("Episodio terminato!")
        obs, _ = env.reset()
        i+=1
env.close()

WARNING! Sprite image ./sprites/.png not found. Using default sprite for this simulation.
WARNING! Sprite image ./sprites/.png not found. Using default sprite for this simulation.
WARNING! Sprite image ./sprites/button.png not found. Using default sprite for this simulation.
WARNING! Sprite image ./sprites/gate_open.png not found. Using default sprite for this simulation.
WARNING! Sprite image ./sprites/gate_close.png not found. Using default sprite for this simulation.


2025-12-25 20:08:08,858	WARNING 3671005061.py:10 -- DeprecationWarning: `compute_single_action` has been deprecated. `Algorithm.compute_single_action` should no longer be used. Get the RLModule instance through `Algorithm.get_module([module ID])`, then compute actions through `RLModule.forward_inference({'obs': [obs batch]})`. This will raise an error in the future!


Episodio terminato!
Episodio terminato!
Episodio terminato!
